In [ ]:
## Data Preparation File
## Group 14

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load in data files
# load in road file
roads = pd.read_csv('_roads3.csv')
# load in bridge file
bridges = pd.read_excel('BMMS_overview.xlsx')

In [ ]:
# Create list of all roadnames
# from road file
road_list = list(roads.road.unique())
road_list.sort()

In [ ]:
# test on the N1 first
def prep_road():
    #road data
    data_fr = roads[roads['road'] == roadname]
    data_fr = data_fr[['road','lrp','lat','lon','chainage']] # subset desired columns
    data_fr.columns = ['Road','LRPName','Lat','Lon','Chainage']
    return data_fr

In [ ]:
def prep_bridge():
    #bridge data
    data_fb = bridges[bridges['road'] == roadname]
    data_fb = data_fb.dropna(subset =['width']) # drop duplicates
    data_fb = data_fb[['road','LRPName','name','chainage','length','condition','lat','lon']] # subset desired columns
    data_fb.columns = ['Road','LRPName','Description','Chainage','Length','Cat','Lat','Lon']
    data_fb = data_fb.sort_values(by = ['Chainage','LRPName'])
    data_fb = data_fb.reset_index(drop = True)
    return data_fb

In [ ]:
# clean duplicate L/R briges before merge
#where LRPName is the same, compare Cat and drop the lower one (if Cat same, drop second one)
def clean_lr(data_fb):
    for i in range(len(data_fb)):
        if i < len(data_fb)-1:
            if data_fb.LRPName.iloc[i,] == data_fb.LRPName.iloc[i+1,]:
                #print("same LRP")
                if data_fb.Cat.iloc[i,] == data_fb.Cat.iloc[i+1,]:
    #                 print("same Cat dropped second entry by default")
                    data_fb.drop(data_fb.index[i+1], inplace = True)
                elif data_fb.Cat.iloc[i,] < data_fb.Cat.iloc[i+1,]:
    #                 print("first Cat greater dropped second entry")
                    #testy.drop(testy.index[[1,3]])
                    data_fb.drop(data_fb.index[i+1], inplace = True)
                elif data_fb.Cat.iloc[i,] > data_fb.Cat.iloc[i+1,]:
    #                 print("second Cat greater dropped first entry")
                    data_fb.drop(data_fb.index[i], inplace = True)
                else:
                    pass
            else:
                pass
        else:
            pass
    return data_fb

In [ ]:
# append files
def join_roadbridge(data_fr,data_fb):
    data = data_fr.append(data_fb, ignore_index = True, sort = False)
    data = data.sort_values(by = ['LRPName']) 
    data = data.reset_index(drop = True)
    return data
    # later, add here instead of calling "n1" everywhere, use a variable (or loop) to apply this to all roads
    # N1.head(50)

In [ ]:
# remove duplicates
#where LRPName repeated from road/brige file, drop road LRP.
def bridge_priority(data):  
    for i in range(len(data)):
        if i < len(data)-1:
            if data.LRPName.iloc[i,] == data.LRPName.iloc[i+1,]:
                #print("same LRP")
                if data.Description.iloc[i,] == "":
    #                 print("des null, is road point, drop it")
                    data.drop(data.index[i], inplace = True)
                elif data.Description.iloc[i,] != "":
    #                 print("des exists, is bridge point, drop other one")
                    data.drop(data.index[i+1], inplace = True)
                else:
                    pass
            else:
                pass
        else:
            pass
    return data

In [ ]:
# set up columns for simio
def simio_columns(data):
    # Set up Node Name column
    data = data.sort_values(by = ['Chainage'])
    data = data.reset_index(drop = True)
    data["Next Node"] = data.LRPName.shift(-1)
    data.loc[data.LRPName.str.contains("LRPE", case = False), "Next Node"] = "PreRome"
    
    # Set up Add On Process column
    data.loc[data.Length.notnull(), "Add On Process"] = "BridgesDelay"
    data.loc[data.Length.isnull(), "Add On Process"] = "BlankProcess"
    # N1.head(10)
    return data

In [ ]:
# Save result
def save_for_simio(data):
    data.to_csv("data_clean.csv", sep=',')

In [ ]:
# make this into loop?
for roadname in road_list: #need file in directory
    prep_road()
    prep_bridge()
    clean_lr(data_fb)
    join_roadbridge(data_fr,data_fb)
    bridge_priority(data)
    simio_columns(data)
    save_for_simio(data)